VANDEN BROECK Grégory
$$$$MAISONNIER Manon

$${\color{green}{\Huge{\textbf{$\fbox{ROCKET FLIGHT}$}}}}$$

$\color{blue}{\underline{\huge{1.\ Introduction}}}$
$$$$ Prenons une fusée de masse $m_s$ avec une masse initiale de carburant $m_p(t = 0)=m_{p0}$.
$$$$ Soit $h$ la hauteur de la fusée par rapport au sol et $v$ sa vitesse. Alors :
$$\frac{dh}{dt}=v$$
$$$$ De plus, si nous définissons $v_e$ comme étant la vitesse d'échappement du carburant, nous pouvons exprimer la force résultante qui s'exerce sur la fusée : 
$$(m_s+m_p)\frac{dv}{dt}=-\ (m_s+m_p)g + \dot{m_p}v_e - \frac{1}{2}\rho v |v| A C_D$$
$$$$ où $\dot{m_p}=\frac{dm_p}{dt}$,$\ A$ est l'aire maximale d'une coupe horizontale de la fusée, $\rho$ la masse volumique de l'air et $C_D$ le coefficient de trainée.
$$$$ Remarquons que la masse du carburant dépend du temps de la manière suivante :
$$m_p = m_{p0} - \int_{0}^{t} \dot{m_p}\ d\tau$$
$$$$ où

$$\dot{m_p}=\left\lbrace \begin{array}{ll} 20 & \mbox{si $t < 5$}\\ 0 & \mbox{sinon} \end{array} \right.$$

$$$$ Nous allons déterminer la hauteur et la vitesse de la fusée par pas de 0,1 s. Pour celà, nous utiliserons la méthode d'Euler qui nous propose l'itération suivante :
$$u_{n+1} = u_n + \Delta{t}. f(u_n)$$
$$$$ où la fonction $f$ décrit la variation de $u_i$ en fonction de $u_i$ dans le temps. Dans notre cas, nous appliquons cette méthode à la hauteur et à la vitesse, en même temps puisqu'ils sont reliés par la relation suivante :

$$\left\lbrace\begin{array}{ll} h_{n+1} = h_n + dt.\frac{dh}{dt}\\v_{n+1} = v_n + dt.\frac{dv}{dt}\\ \end{array}\right. \begin{equation}\Leftrightarrow\left\{\begin{array}{ll} h_{n+1} = h_n + dt.v\\ v_{n+1} = v_n + dt.\left({-g + \frac{\dot{m_p}v_e}{m_s+m_p} - \frac{1}{2}\frac{\rho v |v| A C_D}{m_s+m_p}}\right)\end{array}\right.\end{equation}$$

Dans notre code, ces 2 données seront regroupées dans un vecteur $u_n$.

$\color{blue}{\underline{\huge{2.\ Code}}}$

In [1]:
# Importation de la bibliothèque NumPy pour les opérations sur les vecteurs
import numpy

$\color{red}{\large{\textbf{Données :}}}$

In [2]:
m_s = 50.0 # Masse de la fusée (en kg)
g = 9.81 # Accélération de la pesanteur (en m/s²)
rho = 1.091 # Densité moyenne de l'air (en kg/m³)
r = 0.5 # Rayon maximal d'une coupe horizontale de la fusée (en m)
A = (numpy.pi)*(r**2) # Surface maximum d'une section de la fusée avec un rayon de 0.5 m (en m²)
v_e = 325.0 # Vitesse d'échappement du carburant (en m/s)
C_D = 0.15 # Coefficient de trainée
m_p0 = 100.0 # Masse initiale du carburant au temps t = 0 (en kg)
h0 = 0.0 # Hauteur de la fusée au temps t = 0 (en m)
v0 = 0.0 # Vitesse de la fusée au temps t = 0 (en m/s)
m_p_point_init = 20.0 # Valeur de la dérivée par rapport au temps de la masse du carburant au temps t = 0 en (kg/s)
end = 4.9 # Dernier instant pour lequel la fusée contient du carburant à dt près (en s) (Remarque : En mettant 
          # end = 5, on fait une itération en trop pour v_point et on n'obtient pas la bonne vitesse maximale)

Définissons les fonctions utilisées (dépendantes du temps) pour décrire $\frac{dv}{dt}$, à savoir $\dot{m_p}$ et $m_p$ et répondons à la première question.
$$$$$\color{red}{\large{\textbf{Réponse à la question 1 :}}}$
$$$$$\textbf{1)}$ *At time **t = 3.2s**, what is the mass (in kg) of rocket propellant remaining in the rocket?*

In [3]:
def m_p_point(t) :
    """
    m_p_point correspond à la dérivée de m_p par rapport au temps
    Quand t < 5, m_p_point vaut 20.
    Quand t ≥ 5, m_p_point vaut 0.
    """
    if t < end :
        m_p_point = m_p_point_init
    else :
        m_p_point = 0
    return m_p_point

def integrale(t) :
    """
    Quand t < 5, l'intégrale de 0 à t vaut 20*t.
    Quand t ≥ 5, l'intégrale de 0 à t vaut seulement l'intégrale de 0 à 5 car l'intégrale de 5 à t est nulle 
                 puisque m_p_point vaut 0
    """
    if t < end :
        integrale = m_p_point(t)*t
    else :
        integrale = m_p_point_init*5
    return integrale

def m_p(t) :
    return m_p0 - integrale(t)


print ("Après 3.2 secondes de vol, la fusée ne possède plus que", "%.2f" % round(m_p(3.2),2), "kg de carburant.")

Après 3.2 secondes de vol, la fusée ne possède plus que 36.00 kg de carburant.


Préparons-nous à faire nos itérations.

In [4]:
T = 40.0 # Temps maximal pour lequel on va chercher h et v (en s)
dt = 0.1 # Intervalle de temps entre chaque valeur cherchée (en s)
N = int(T/dt)+1 # Nombre de valeurs que nous allons chercher pour u = (h,v)
t = numpy.linspace(0.0, T, N) # Toutes les valeurs t[n] que prend t selon u_n = (h_n,v_n)
u = numpy.empty((N, 2)) # On crée un vecteur u de N*2 éléments que l'on va remplir de u_n = (h_n,v_n)
u[0] = numpy.array([h0, v0])
t0 = 0 # Temps initial (en s)

Définissons $f(u,t) = \frac{du}{dt}$ où $u = (h,v)$ grâce à : $$\frac{dv}{dt}=-g + \left(\frac{\dot{m_p}v_e}{m_s+m_p}\right) - \left(\frac{1}{2}\frac{\rho v |v| A C_D}{m_s+m_p}\right)$$ et $$\frac{dh}{dt}=v$$

In [5]:
def f(u,t) :
    """
    La fonction qui décrit la variation de u_i à chaque instant t[i]
    """
    h = u[0]
    v = u[1]
    v_point = -g + ((m_p_point(t)*v_e)/(m_s + m_p(t))) - ((rho*v*abs(v)*A*C_D)/(2.0*(m_s + m_p(t))))
    return numpy.array([v, v_point])

Méthode d'Euleur : $u_{n+1}=u_n + \Delta{t}.f(u_n)$

In [6]:
def euler_step(u,f,t) :
    """
    L'itération de la méthode d'Euler prenant u_n et retournant u_{n+1}
    """
    return u + dt*f(u,t)

In [7]:
for n in range(N-1) :
    u[n+1] = euler_step(u[n],f,t0) # Dans notre vecteur (de vecteurs) u défini 3 cellules plus haut, on place 
                                   # les valeurs de u_{n+1} définies grâce à la méthode d'Euler
    t0 = t0 + dt

A présent, nous avons trouvé nos valeurs. Nous pouvons maintenant répondre aux questions suivantes.
$$$$$\color{red}{\large{\textbf{Réponse aux questions 2, 3 et 4 :}}}$ 
$$$$$\textbf{2)}$ *What is the maximum speed of the rocket in m/s?*
$$$$$\textbf{3)}$ *At what time does this occur (in seconds)?*
$$$$$\textbf{4)}$ *What is the altitude at this time (in meters)?*

In [8]:
v_max = 0 # Vitesse maximale de la fusée (en m/s)
h_v_max = 0 # Hauteur pour laquelle la vitesse de la fusée est maximale (en m)
i = 0
for vect in u :
    while vect[1] > v_max :
        v_max = vect[1]
        h_v_max = vect[0]
        i = i+1
temps = i*dt 


print ("La vitesse maximale de la fusée est de", "%.2f" % round(v_max,2), "m/s. Elle atteint cette vitesse en", "%.2f" % round(temps,2), "secondes et son altitude à cet instant est de", "%.2f" % round(h_v_max,2), "m.")

La vitesse maximale de la fusée est de 232.11 m/s. Elle atteint cette vitesse en 5.00 secondes et son altitude à cet instant est de 523.52 m.


$\color{red}{\large{\textbf{Réponse aux questions 5 et 6 :}}}$ 
$$$$$\textbf{5)}$ *What is the rocket's maximum altitude during flight (in meters)?*
$$$$$\textbf{6)}$ *At what time (in seconds) does this occur?*

In [9]:
h_max = 0 # Hauteur maximale de la fusée lors de son vol (en m)
i = 0
for vect in u :
    while vect[0] > h_max :
        h_max = vect[0]
        i = i+1
temps = i*dt


print ("La hauteur maximale de la fusée lors de son vol est de", "%.2f" % round(h_max,2), "m. Elle atteint cette hauteur en", "%.2f" % round(temps,2), "secondes.")

La hauteur maximale de la fusée lors de son vol est de 1334.18 m. Elle atteint cette hauteur en 15.60 secondes.


$\color{red}{\large{\textbf{Réponse aux questions 7 et 8 :}}}$
$$$$$\textbf{7)}$ *At what time (in seconds) does the rocket impact the ground?*
$$$$$\textbf{8)}$ *What is the velocity of the rocket (in m/s) at time of impact?*
$$$$$\color{orange}{\underline{\textit{Remarque}}}$ : Pour répondre à la question $8)$, nous avons utilisé le fait que quand la hauteur $(u[0])$ est positive, la fusée n'a pas encore percuté le sol et quand elle est négative, la fusée est "en dessous du sol" et donc, physiquement, l'impact a eu lieu.

In [10]:
v_imp = numpy.empty((30, 1)) # On crée un tableau (à proprement parlé "array" donc un vecteur) pour ranger 
                             # tout les u[1] après l'impact au sol
i = 0
j = 1
for vect in u :
    if vect[0] == abs(vect[0]) :
        j = j+1
    else :
        v_imp[i] = vect[1]
        i = i+1
temps = j*dt 


print("Après", "%.2f" % round(temps,2), "secondes de vol, l'impact de la fusée avec le sol a lieu. A cet instant, la fusée possède une vitesse de", v_imp[0], "m/s.")

Après 37.20 secondes de vol, l'impact de la fusée avec le sol a lieu. A cet instant, la fusée possède une vitesse de [-86.00683498] m/s.
